In [13]:
pip install torch torchvision opencv-python open3d timm matplotlib pillow

Note: you may need to restart the kernel to use updated packages.


In [14]:
import cv2
import torch
import numpy as np
import open3d as o3d
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import matplotlib.pyplot as plt

In [19]:
img_path = "tomandjerry.jpeg"
img_bgr = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_pil = Image.fromarray(img_rgb)

# Convert PIL to NumPy array
img_np = np.array(img_pil)

# Load MiDaS
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid", trust_repo=True)
midas.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform

# Transform and predict
input_batch = midas_transforms(img_np)
with torch.no_grad():
    prediction = midas(input_batch)
    depth = prediction.squeeze().cpu().numpy()

# Combine with original RGB
depth_resized = cv2.resize(depth, (img_rgb.shape[1], img_rgb.shape[0]))
encoded_matrix = np.dstack((img_rgb, depth_resized))  # (H, W, 4)

Using cache found in /Users/rexouyang/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /Users/rexouyang/.cache/torch/hub/intel-isl_MiDaS_master


In [23]:
#turn this into 3d cloud
H, W, _ = encoded_matrix.shape
flattened = encoded_matrix.reshape(-1, 4)  # shape (H*W, 4)

points_xyz = []
colors_rgb = []

for idx, (r, g, b, d) in enumerate(flattened):
    i = idx // W  # row
    j = idx % W   # column
    x, y, z = j, i, d  # simple mapping — or use camera intrinsics if available

    points_xyz.append([x, y, z])
    colors_rgb.append([r/255.0, g/255.0, b/255.0])


In [ ]:
#make clusters
